### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 以句點為分隔拆句子

In [1]:
import json
import nltk

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
        
    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [14]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 48901
print(ave)

32.537698615570235


In [19]:
for tk,lab in docs[1]:
    print(tk,lab)

每 O
于 O
夏 O
日 O
洗 O
澡 O
后 O
即 O
寒 B-S
热 I-S
不 I-S
适 E-S
， O
自 O
感 O
发 O
热 O
而 O
又 O
需 O
披 O
衣 O
裹 O
护 O


In [5]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [6]:
print(len(data))

48901


In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print(len(X_train))
print(len(X_test))

39120
9781


In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 30152
Seconds required: 3.047

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 295747.614333
Feature norm: 1.000000
Error norm: 80927.730835
Active features: 30047
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 1.100

***** Iteration #2 *****
Loss: 293948.506837
Feature norm: 1.003833
Error norm: 78220.103724
Active features: 28412
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.561

***** Iteration #3 *****
Loss: 261537.060149
Feature norm: 1.258883
Error norm: 113309.466763
Active features: 20749
Line search trials: 1
Line search step: 1.000000
Seconds required

***** Iteration #43 *****
Loss: 38563.681703
Feature norm: 53.272419
Error norm: 7712.533788
Active features: 23589
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.559

***** Iteration #44 *****
Loss: 38160.639653
Feature norm: 55.457228
Error norm: 4422.993244
Active features: 23680
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.562

***** Iteration #45 *****
Loss: 37797.115852
Feature norm: 57.758080
Error norm: 5198.573556
Active features: 23570
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.562

***** Iteration #46 *****
Loss: 37490.840173
Feature norm: 59.955622
Error norm: 7614.138678
Active features: 23578
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.560

***** Iteration #47 *****
Loss: 37195.346063
Feature norm: 61.981746
Error norm: 9073.655386
Active features: 23418
Line search trials: 1
Line search step: 1.000000

***** Iteration #85 *****
Loss: 33731.240695
Feature norm: 101.406329
Error norm: 6854.431140
Active features: 21688
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.565

***** Iteration #86 *****
Loss: 33711.066280
Feature norm: 101.851861
Error norm: 6789.041798
Active features: 21649
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.557

***** Iteration #87 *****
Loss: 33687.507967
Feature norm: 102.471013
Error norm: 6294.185080
Active features: 21666
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.563

***** Iteration #88 *****
Loss: 33667.864877
Feature norm: 102.793557
Error norm: 4362.757753
Active features: 21713
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.559

***** Iteration #89 *****
Loss: 33648.731127
Feature norm: 103.210679
Error norm: 4390.540769
Active features: 21699
Line search trials: 1
Line search step: 1.0

***** Iteration #128 *****
Loss: 33238.382745
Feature norm: 115.511256
Error norm: 2878.084201
Active features: 21226
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.562

***** Iteration #129 *****
Loss: 33233.215701
Feature norm: 115.773736
Error norm: 4065.194918
Active features: 21185
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.561

***** Iteration #130 *****
Loss: 33226.023131
Feature norm: 115.983952
Error norm: 2904.258194
Active features: 21182
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.563

***** Iteration #131 *****
Loss: 33221.136034
Feature norm: 116.233545
Error norm: 4043.259670
Active features: 21180
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.563

***** Iteration #132 *****
Loss: 33213.801925
Feature norm: 116.419198
Error norm: 2732.906166
Active features: 21185
Line search trials: 1
Line search step

***** Iteration #168 *****
Loss: 33026.358208
Feature norm: 122.440350
Error norm: 2587.874046
Active features: 20928
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.556

***** Iteration #169 *****
Loss: 33022.254669
Feature norm: 122.618743
Error norm: 3626.303662
Active features: 20931
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.557

***** Iteration #170 *****
Loss: 33016.550948
Feature norm: 122.738872
Error norm: 2501.339682
Active features: 20895
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.561

***** Iteration #171 *****
Loss: 33012.918658
Feature norm: 122.916539
Error norm: 3634.941423
Active features: 20903
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.558

***** Iteration #172 *****
Loss: 33007.216492
Feature norm: 123.039657
Error norm: 2401.079205
Active features: 20903
Line search trials: 1
Line search step

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

坚 (O)
持 (O)
服 (O)
抗 (O)
真 (O)
菌 (O)
进 (O)
口 (O)
药 (O)
“ (O)
伏 (O)
立 (O)
康 (O)
唑 (O)
” (O)
半 (O)
年 (O)
， (O)
不 (O)
仅 (O)
未 (O)
能 (O)
控 (O)
制 (O)
， (O)
近 (O)
期 (O)
以 (O)
来 (O)
发 (O)
作 (O)
反 (O)
而 (O)
更 (O)
加 (O)
频 (O)
繁 (O)
， (O)
发 (O)
时 (O)
体 (O)
温 (O)
常 (O)
达 (O)
4 (O)
0 (O)
. (O)
5 (O)
℃ (O)


In [20]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

             precision    recall  f1-score   support

          O       0.97      0.99      0.98    307289
      B-SYM       0.59      0.46      0.52      4414
      I-SYM       0.52      0.33      0.41      5369
      E-SYM       0.60      0.47      0.53      4396

avg / total       0.96      0.96      0.96    321468

